# DSPy RAG Agent Demo

This notebook demonstrates how to use the DSPy-based RAG agent with FAISS retrieval to answer questions about the Chess Fundamentals PDF.

In [1]:
import os
import sys
import dspy
from dotenv import load_dotenv

# Add src to path so we can import hack module
# sys.path.insert(0, os.path.abspath('../src'))

from hack.rag_agent import create_agent

/Users/yuriyarabskyy/Library/Caches/pypoetry/virtualenvs/hack-large-text-ASlkIg2G-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

print("✓ API key loaded")

✓ API key loaded


In [3]:
# Configure DSPy with OpenAI
lm = dspy.LM(model="openai/gpt-4o-mini", api_key=openai_api_key)
dspy.configure(lm=lm)

print("✓ DSPy configured with OpenAI GPT-4o-mini")

✓ DSPy configured with OpenAI GPT-4o-mini


In [4]:
# Initialize the agent with FAISS retriever
print("Initializing agent with FAISS retriever...")
agent = create_agent(
    faiss_index_path="chess_pdf.faiss",
    workspace_json_path="workspace_with_embeddings.json"
)
print("✓ Agent initialized")

Initializing agent with FAISS retriever...
Loaded FAISS index with 1941 vectors
Loaded 1941 blocks from workspace
✓ Agent initialized


## Example 1: Opening Strategy

In [5]:
question1 = "What are the key principles for opening moves in chess?"
print(f"Question: {question1}\n")

result1 = agent.forward(question1)

print("Answer:")
print(result1["answer"])
print("\nCitations:")
print(result1["citations"])

2025/10/02 14:34:34 WARNING dspy.primitives.module: Calling module.forward(...) on WorkspaceAgent directly is discouraged. Please use module(...) instead.


Question: What are the key principles for opening moves in chess?

Answer:
The key principles for opening moves in chess include the importance of developing pieces over advancing pawns, particularly in the early stages of the game. Beginners often make the mistake of prioritizing pawn moves, such as moving the pawns to the rook's third rank, which can hinder their overall development. Instead, players should focus on controlling the center of the board, developing knights and bishops, and ensuring king safety through castling. Understanding pawn structures can also provide insights into effective opening strategies and transitions into the middle game.

Citations:
[
    {
        "page": 52,
        "section": "FURTHER OPENINGS AND MIDDLE-GAMES > 31. SOME SALIENT POINTS ABOUT PAWNS",
        "text": "Before going back to the discussion of openings and middle-game positions, it might be well to bear in mind a few facts concerning Pawn positions which will no doubt help to understand ce

## Example 2: Endgame Strategy

In [11]:
question2 = "How should I approach the endgame in chess?"
print(f"Question: {question2}\n")

result2 = agent.forward(question2)

print("Answer:")
print(result2["answer"])
print("\nCitations:")
print(result2["citations"])

2025/10/02 14:54:48 WARNING dspy.primitives.module: Calling module.forward(...) on WorkspaceAgent directly is discouraged. Please use module(...) instead.


Question: How should I approach the endgame in chess?

Answer:
To approach the endgame in chess effectively, it's essential to focus on a few key principles. First, understanding the specific endgame scenarios, such as those involving two rooks and pawns, can significantly enhance your strategy. Studying master games that feature these endings will provide valuable insights into the proper methods to employ in similar situations. 

Additionally, it's crucial to apply general endgame principles, such as activating your king, promoting pawns, and coordinating your pieces effectively. The endgame often requires precise calculation and an understanding of the fundamental concepts of piece activity and pawn structure. By practicing these strategies and analyzing various endgame positions, you can improve your overall endgame play and increase your chances of success.

Citations:
[
    {"page": 46, "section": "EXAMPLE 59", "text": "Following our idea that the best way to learn endings as wel

In [10]:
result2

{'answer': "To approach the endgame in chess effectively, it's essential to focus on a few key principles. First, understanding the specific endgame scenarios, such as those involving two rooks and pawns, can significantly enhance your strategy. Studying master games that feature these endings will provide valuable insights into the proper methods to employ in similar situations. \n\nAdditionally, it's crucial to apply general endgame principles, such as activating your king, promoting pawns, and coordinating your pieces effectively. The endgame often requires precise calculation and an understanding of the fundamental concepts of piece activity and pawn structure. By practicing these strategies and analyzing various endgame positions, you can improve your overall endgame play and increase your chances of success.",
 'citations': '[\n    {"page": 46, "section": "EXAMPLE 59", "text": "Following our idea that the best way to learn endings as well as openings is to study the games of the 

## Example 3: Pawn Structure

In [ ]:
question3 = "What is the importance of pawn structure in chess?"
print(f"Question: {question3}\n")

result3 = agent.forward(question3)

print("Answer:")
print(result3["answer"])
print("\nCitations:")
print(result3["citations"])

## Example 4: Specific Tactical Question

In [ ]:
question4 = "What does Capablanca say about the value of pieces?"
print(f"Question: {question4}\n")

result4 = agent.forward(question4)

print("Answer:")
print(result4["answer"])
print("\nCitations:")
print(result4["citations"])

## Example 5: Custom Question

Try your own question!

In [ ]:
# Your custom question here
custom_question = "How do I develop my pieces effectively?"
print(f"Question: {custom_question}\n")

custom_result = agent.forward(custom_question)

print("Answer:")
print(custom_result["answer"])
print("\nCitations:")
print(custom_result["citations"])

## Inspecting the RAG Pipeline

Let's look at what happens behind the scenes in the agent's forward pass.

In [ ]:
# Test the retriever directly
from hack.retriever import FaissRetriever

retriever = FaissRetriever(
    faiss_index_path="chess_pdf.faiss",
    workspace_json_path="workspace_with_embeddings.json"
)

# Search for relevant passages
search_query = "tactics and strategy"
results = retriever.search_text(search_query, k=3)

print(f"Search query: '{search_query}'\n")
print(f"Found {len(results)} results:\n")

for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"  Page: {result['page']}")
    print(f"  Similarity: {result['similarity']:.4f}")
    print(f"  Text: {result['content'][:200]}...")
    print()

## Understanding the Agent Architecture

The RAG agent uses a 3-step pipeline:

1. **Query Understanding** - Expands the question into search terms
2. **Retrieval** - Searches FAISS index for relevant passages
3. **Evidence Selection** - Selects the most relevant evidence
4. **Answer Synthesis** - Generates answer with citations

Each step uses DSPy signatures and the LM to process information.

In [ ]:
# Let's test just the query understanding step
from hack.models.rag_models import QueryUnderstanding

understand = dspy.Predict(QueryUnderstanding)
q_resp = understand(question="What are common chess mistakes beginners make?")

print("Query Understanding:")
print(f"  Original question: What are common chess mistakes beginners make?")
print(f"  Search terms: {q_resp.search_terms}")
print(f"  Search plan: {q_resp.search_plan}")

## Interactive Q&A Session

Run this cell multiple times with different questions!

In [ ]:
def ask_question(question: str):
    """Helper function to ask a question and display results nicely."""
    print(f"\n{'='*80}")
    print(f"Q: {question}")
    print(f"{'='*80}\n")
    
    result = agent.forward(question)
    
    print("Answer:")
    print(result["answer"])
    print("\n" + "-"*80)
    print("Citations:")
    print(result["citations"])
    print("="*80)
    
    return result

# Try it out!
ask_question("What does Capablanca teach about rook endgames?")

## Performance Analysis

In [ ]:
import time

# Measure query time
test_question = "How do I checkmate with a queen and king?"

start = time.time()
result = agent.forward(test_question)
end = time.time()

print(f"Question: {test_question}")
print(f"\nTime taken: {end - start:.2f} seconds")
print(f"\nAnswer: {result['answer'][:200]}...")

## Summary

This notebook demonstrated:
- Setting up the DSPy RAG agent with FAISS retrieval
- Asking various chess-related questions
- Getting answers with citations
- Inspecting the retrieval pipeline
- Understanding the agent architecture

The agent successfully retrieves relevant passages from the Chess Fundamentals PDF and synthesizes answers with proper citations!